In [1]:
import numpy as np
from tensor2tensor.utils import rouge
from tensorflow.keras.preprocessing import sequence
import sentencepiece as spm
import json

In [2]:
sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.t5.model')

True

In [3]:
with open('output-base.json') as fopen:
    predicted_base = json.load(fopen)
    
len(predicted_base)

10000

In [4]:
with open('output-small.json') as fopen:
    predicted_small = json.load(fopen)
    
len(predicted_small)

10000

In [5]:
with open('test-set-cnn.json') as fopen:
    data = json.load(fopen)
    
len(data['Y'])

10000

In [6]:
data['Y'][0], predicted_base[0], predicted_small[0]

("British Airways telah membeli pembekal perniagaan Perancis L'Avion untuk Ps 54 juta. Syarikat penerbangan itu akan disepadukan ke E. U. -U. S. Anak syarikat Openskies. Jika diluluskan, integrasi penuh dijangka pada awal tahun 2009",
 "British Airways telah bersetuju untuk membeli L'Avion, syarikat penerbangan kelas perniagaan Perancis. Perjanjian $ 108 juta adalah pemergian syarikat penerbangan serba ada yang terakhir berikutan kematian Maxjet, Eos dan Silverjet. Openskies akan meningkatkan kekerapan perkhidmatan Orly-Jfk kepada tiga penerbangan harian",
 "British Airways membeli L'Avion, syarikat penerbangan kelas perniagaan Perancis. Syarikat penerbangan itu dilancarkan pada Januari 2009 dan menggunakan 77 kakitangan di Paris. Perjanjian $ 108 juta menandakan pemergian syarikat penerbangan serba ada yang terakhir selepas kematian Maxjet")

In [7]:
def calculate_rouges(predicted, batch_y, n_size = 2):
    batch_y = [sp_model.EncodeAsIds(row) for row in batch_y]
    predicted = [sp_model.EncodeAsIds(row) for row in predicted]
    maxlen = max(max(len(row) for row in batch_y), max(len(row) for row in predicted))
    batch_y = sequence.pad_sequences(batch_y, padding = 'post', maxlen = maxlen)
    predicted = sequence.pad_sequences(predicted, padding = 'post', maxlen = maxlen)
    
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False, maxlen = maxlen)
    rouges = []
    for i in range(b.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        score = rouge.rouge_n([a], [p], n = n_size)
        rouges.append(score)
    return np.mean(rouges)

def calculate_rouge_l(predicted, batch_y):
    batch_y = [sp_model.EncodeAsIds(row) for row in batch_y]
    predicted = [sp_model.EncodeAsIds(row) for row in predicted]
    maxlen = max(max(len(row) for row in batch_y), max(len(row) for row in predicted))
    batch_y = sequence.pad_sequences(batch_y, padding = 'post', maxlen = maxlen)
    predicted = sequence.pad_sequences(predicted, padding = 'post', maxlen = maxlen)
    
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False, maxlen = maxlen)
    rouges = []
    for i in range(b.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        score = rouge.rouge_l_sentence_level([a], [p])
        rouges.append(score)
    return np.mean(rouges)

In [8]:
calculate_rouges(predicted_base, data['Y'], n_size = 1)

0.3410321

In [9]:
calculate_rouges(predicted_small, data['Y'], n_size = 1)

0.3385415

In [10]:
calculate_rouges(predicted_base, data['Y'], n_size = 2)

0.1499447

In [11]:
calculate_rouges(predicted_small, data['Y'], n_size = 2)

0.14588113

In [12]:
calculate_rouge_l(predicted_base, data['Y'])

0.23655131

In [13]:
calculate_rouge_l(predicted_small, data['Y'])

0.23528054